In [44]:
from PIL import Image
import imdb
from io import BytesIO
import requests
import pickle
from pathlib import Path
ia = imdb.IMDb()

NUMBER_OF_TOP_ACTORS = 10


def save_movie(movie, movies_dir):
  #save to pickle
  with open(movies_dir/f'{movie["title"]}-{movie.getID()}.pkl', 'wb') as f:
    pickle.dump(movie, f)
#     print('saved', f'{movie["title"]}-{movie.getID()}.pickle')

def get_actors_images(movie, movies_dir):
  cast = movie.get('cast')
  if cast:
      top_actors = cast[0:NUMBER_OF_TOP_ACTORS]

      for a in top_actors:
        actor_id = a.getID()
        actor_dir = movies_dir/f'{a["name"]}-{actor_id}'

        actor_dir.mkdir(exist_ok=True)
        if not (actor_dir/'actor.pkl').exists():
            person = ia.get_person(actor_id)

            try:
                url = person['full-size headshot']

                if(url):
                  response = requests.get(url)
                  img = Image.open(BytesIO(response.content))
            
                  img.save(actor_dir/'0.jpg')
    #               print('saved', a['name'])
            except KeyError:
                pass
            except OSError:
                print("OSError")
            with open(f'{actor_dir}/actor.pkl', 'wb') as f:
                pickle.dump(person, f)

In [10]:
data_path = Path("data")
movies_path = data_path / "movie_posters_filtered.pkl"

In [11]:
with movies_path.open("rb") as f:
    df = pickle.load(f)

In [18]:
df["id"] = df["tconst"].str.strip("tt")

In [37]:
movies_dir = data_path/'movies'
def get_movie_actors(movie_id):
    try:
        if len(list(movies_dir.glob(f"*{movie_id}.pkl"))) == 0:
            movie = ia.get_movie(movie_id)
            get_actors_images(movie,data_path/"actors")
            save_movie(movie, movies_dir)
    except imdb.IMDbDataAccessError:
        print("IMDbDataAccessError")


In [ ]:
from tqdm import tqdm_notebook as tqdm
import concurrent

max_workers = 8
res = []
movie_ids = list(df["id"])
with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
    for movie_id in tqdm(executor.map(get_movie_actors, movie_ids), total=len(movie_ids)):
         pass

OSError
OSError
OSError
OSError
OSError


2020-11-24 16:14:49,874 CRITICAL [imdbpy] /storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm0001992/', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out',)},); kwds: {}
Traceback (most recent call last):
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/site-packages/imdb/parser/http/__init__.py", line 220, in retrieve_unicode
    response = uopener.open(url)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/storage/homedir/dima/miniconda3/envs/subs2graph/lib/python3.6/urllib/request.py", line 504, in _call_chain
    res

In [38]:
for movie_id in tqdm(movie_ids):
   get_movie_actors(movie_id)

KeyboardInterrupt: 